In [38]:
import tweepy
import csv
import pandas as pd
import numpy as np
import string

df = pd.read_csv('project_data.csv')

# Make personal_twitter handles of those who have an empty cell into the value held in the official_twitter column
df.loc[df['personal_twitter'].isnull(),'personal_twitter'] = df['official_twitter']
# We'll drop any congressman/woman who doesn't have niether a personal nor official twitter (Andrew S. Clyde)
df.dropna(subset=['personal_twitter'], inplace=True)

congressmen_list = list(df['personal_twitter'])

auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret_Key)
auth.set_access_token(Access_Key, Access_Secret_Key)
api = tweepy.API(auth, wait_on_rate_limit=True)

def create_csv(filename, usernames):
    csvfile = open(filename, 'w')
    c = csv.writer(csvfile)
    c.writerow([ "Bio", 'Follower Count' ])
    for name in usernames:
        user_info = get_userinfo(name)
        c.writerow(user_info)
    csvfile.close()

def get_userinfo(name):
    try:
        user = api.get_user(screen_name = name)
        user_info = [user.description.encode('utf-8'), user.followers_count]
        return user_info
    except:
        user_info = ['', np.NaN]
        return user_info

NameError: name 'Consumer_Key' is not defined

In [2]:
create_csv('congressmen_descriptions.csv', congressmen_list)

In [39]:
descriptions = pd.read_csv('congressmen_descriptions.csv')
descriptions['Bio'] = descriptions['Bio'].str[2:-1]
df = df.reset_index()
descriptions['names'] = df['full_name']
result = pd.concat([df, descriptions], axis=1)

In [41]:
punc = string.punctuation.replace('-', '').replace('/', '')
del result['index']
del result['names']
result['Bio'] = result['Bio'].str.replace('[{}]'.format(punc), ' ').str.lower()
result.to_csv('project_data_complete.csv')